## TensorFlow TensorBoard监控指标和高维向量可视化

选择环境：Anaconda Python 3.6.6  
安装Tensorflow：Python 3.6环境下运行pip install --upgrade --ignore-installed tensorflow  
参考书籍：《TensorFlow实战Google深度学习框架（第2版）》  
ipynb格式：点击阅读原文github

## 11.3 监控指标可视化

除GRAPHS以外，TensorBoard界面中还提供SCALARS、IMAGES、AUDIO、DISTRIBUTIONS、HISTOGRAMS和TEXT六个界面来可视化其他的监控指标。以下程序展示了如何将TensorFlow程序运行时的信息输出到TensorBoard日志文件中。

该段代码不能运行在JupyterNotebook，可以使用Spyder运行。

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

SUMMARY_DIR = "log/11.3.log"
BATCH_SIZE = 100
TRAIN_STEPS = 3000

# 1. 生成变量监控信息并定义生成监控信息日志的操作。
# 其中var给出了需要记录的张量，name给出了在可视化结果中显示的图发名称，这个名称一般与变量名一致。
def variable_summaries(var, name):
    # 将生成监控信息的操作放到统一命名空间下
    with tf.name_scope('summaries'):
        # 通过tf.summary.histogram函数记录张量中元素的取值分布。对于给出的图表
        # 名称和张量，tf.summary.histogram函数会生成一个Summary protocol buffer。
        # 将Summary写入TensorBoard日志文件后，在HISTOGRAMS栏和DISTRIBUTION栏下
        # 都会出现对应名称的图表。和TensorFlow中其他操作类似，
        # tf.summary.histogram函数不会立刻被执行，只有当sess.run函数明确调用这
        # 个操作时，TensorFlow才会具正生成并输出Summary protocol buffer。
        # 下文将更加详细地介绍如何理解HISTOGRAMS栏和DISTRIBUTION栏下的信息。
        tf.summary.histogram(name, var)
        
        # 计算变量的平均值，并定义生成平均值信息日志的操作。记录变量平均值信息的日志标签名
        # 为'mean/'＋name，其中mean为命名空间，/是命名空间的分隔符，从图11.14中可以看到，
        # 在相同命名空间中的监控指标会被整合到同一栏中；name则给出了当前监控指标属于哪一个变量。
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean/' + name, mean)
        
        # 计算变量的标准差，并定义生成其日志的操作
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev/' + name, stddev)  
        

# 2. 生成一层全链接的神经网络。
def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    # 将同一层神经网络放在一个统一的命名空间下
    with tf.name_scope(layer_name):
        # 声明神经网络边上的权重，并调用生成权重监控信息日志的函数。
        with tf.name_scope('weights'):
            weights = tf.Variable(tf.truncated_normal([input_dim, output_dim], stddev=0.1))
            variable_summaries(weights, layer_name + '/weights')
            
        # 声明神经网络的偏置项，并调用生成偏置项监控信息日志的函数。    
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.constant(0.0, shape=[output_dim]))
            variable_summaries(biases, layer_name + '/biases')
            
        with tf.name_scope('Wx_plus_b'):
            preactivate = tf.matmul(input_tensor, weights) + biases
            # 记录神经网络输出节点在经过激活的数之前的分布。
            tf.summary.histogram(layer_name + '/pre_activations', preactivate)
        activations = act(preactivate, name='activation')        
        
        # 记录神经网络输出节点在经过激活函数之后的分布。在图11.17中，对于layer1，因
        # 为使用了ReLU函数作为激活函数，所以所有小于0的值部被设为了0。于是在激活后
        # 的layer1/activations图上所有的值都是大于0的。而对于layer2，因为没有使用
        # 激活函数，所以layer2/activations和layer2/pre_activations一样。
        tf.summary.histogram(layer_name + '/activations', activations)
        return activations
    

# 3. 主函数
def main():
    mnist = input_data.read_data_sets("../../datasets/MNIST_data", one_hot=True)
    # 定义输入
    with tf.name_scope('input'):
        x = tf.placeholder(tf.float32, [None, 784], name='x-input')
        y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')

    # 将输入向量还原成图片的像素矩阵，并通过tf.summary.image函数将当前的图片信息写入日志的操作
    with tf.name_scope('input_reshape'):
        image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
        tf.summary.image('input', image_shaped_input, 10)

    hidden1 = nn_layer(x, 784, 500, 'layer1')
    y = nn_layer(hidden1, 500, 10, 'layer2', act=tf.identity)
    
    # 计算交叉熵并定义生成交叉熵监控日志的操作
    with tf.name_scope('cross_entropy'):
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y, labels=y_))
        tf.summary.scalar('cross_entropy', cross_entropy)

    with tf.name_scope('train'):
        train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

    # 当前模型在当前给定数据上的正确率，并定义生成正确率监控日志的操作。如果在sess.run
    # 时给定的数据训练batch，那么得到的正确率就是在这个训练batch上的正确率；如果给定的
    # 数据为验证或者测试数据，那么得到的正确率就是在当前模型在验证或者测试数据上的正确率。
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('accuracy', accuracy)

    # 和TensorFlow其他操作类似，tf.summary.scalar、tf.summary.histogram和tf.summary.image
    # 函数都不会立即执行，需要通过sess.run来明确调用这些函数。因为程序中定义的写日志操作
    # 比较多，一一调用非常麻烦，所以TensorFlow提供了tf.summary.merge_all函数来整理所有的
    # 日志生成操作。在TensorFlow程序执行的过程中只要运行这个操作就可以将代码中定义的所有
    # 日志生成操作执行一次，从而将所有日志文件。
    merged = tf.summary.merge_all()

    with tf.Session() as sess:
        # 初始化写日志的writer，并将当前TensorFlow计算图写入日志。
        summary_writer = tf.summary.FileWriter(SUMMARY_DIR, sess.graph)
        tf.global_variables_initializer().run()

        for i in range(TRAIN_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            # 运行训练步骤以及所有的日志生成操作，得到这次运行的日志。
            summary, _ = sess.run([merged, train_step], feed_dict={x: xs, y_: ys})
            # 将得到的所有日志写入日志文件，这样TensorBoard程序就可以拿到这次运行所对应的
            # 运行信息。
            summary_writer.add_summary(summary, i)

    summary_writer.close()
    
    
if __name__ == '__main__':
    main()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../datasets/MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../datasets/MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../datasets/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../../datasets/MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


可以看出，除了GRAPHS之外，Tensorboard中的每一栏对应了TensorFlow中一种日志生成函数。

**1. SCALARS**

展示了样例程序中通过`tf.summary.scalar`函数生成的所有标量监控信息。和变量的命名空间类似，TensorBoard也会根据监控指标的名称进行分组。

和TensorFlow计算图可视化结果不同的是，**SCALARS、IMAGES、AUDIO、TEXT、HISTOGRAMS和DISTRUBUTIONS栏只会对最高层的命名空间进行整合，单击展开后将看到该命名空间下的所有监控指标**。

在每一个监控指标的左下角有一个小方框，单击这个方框可以得到放大后的图片。在训练神经网络时，通过TensorBoard监控神经网络中变量取值的变化、模型在训练batch上的损失函数大小以及学习率的变化等信息可以更加方便地掌握模型的训练情况。

**2. IMAGES**

展示了通过TensorBoard可视化当前轮训练使用的图像信息。通过这个界面可以大致看出数据随机打乱的效果。因为TensorFlow程序和TensorBoard可视化界面可以同时运行，所以从TensorBoard上可以实时看到TensorFlow程序中最新使用的训练或者测试图像。

**3. DISTRIBUTIONS**

DISTRIBUTIONS一栏提供了对张量取值分布的可视化界面。可以直观地观察到不同层神经网络中参数的取值变化。

**4. HISTOGRAMS**

为了更加清晰地展示参数取值分布和训练选代轮数之间的关系，TensorBoard提供了HISTOGRAMS视图。与DISTRIBUTIONS效果图不同，**HISTOGRAMS中不同轮数中参数的取值是通过不同的平面来表示的**。颜色越深的平面表示迭代轮数越小的取值分布，比如最上面的比较尖的平面表示训练一轮之后的bias参数取值分布。因为bias是通过全0矩阵初始化的，于是在第一轮时取值都集中在0附近。最前面比较浅的平面表示迭代轮数较大时的参数取值分布。可以看到bias的取值分布越来越接近平均分布。

在HISTOGRAMS视图左侧有一个“OVERLAY”选项，和默认的OFFSET视图类似，在OVERLAY视图中颜色越深的表示迭代轮数越小。但是比较尖的曲线看上去颜色比较浅，而比较靠近平均分布的曲线反而比较深。这是因为有更多的曲线靠近平均分布，所以合在一起就比较深了。当把鼠标移到某一条曲线上时这一条曲线就会变黑，而且迭代轮数的信息会显示在鼠标附近。

## 11.4 高维向量可视化

TensorBoard提供了PROJECTOR界面来可视化高维向量之间的关系。比如在图像迁移学习中可以将一组目标问题的图片通过训练好的卷积层得到瓶颈层，这些瓶颈层向量就是多个高维向量。如果在目标问题图像数据集上同一种类的图片在经过卷积层之后得到的瓶颈层向量在空间中比较接近，那么这样迁移学习得到的结果就有可能会更好。类似地，在训练单词向量时，如果语义相近的单词所对应的向量在空间中的距离也比较接近的话，那么自然语言模型的效果也有可能会更好。

为了在PROJECTOR中更好地展示MNIST图片信息以及每张图片对应的真实标签，PROJECTOR要求用户准备一个sprite图像和一个tsv文件给出每张图片对应的标签信息（所谓sprite图像就是将一组图片组合成一整张大图片）。以下代码给出了如何使用MNIST测试数据生成PROJECTOR所需要的这两个文件。

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import numpy as np
import os
from tensorflow.examples.tutorials.mnist import input_data

# PROJECTOR需要的日志文件名和地址相关参数
LOG_DIR = 'log/high_dim'
SPRITE_FILE = 'mnist_sprite.jpg'
META_FIEL = "mnist_meta.tsv"

# 使用给出的MNIST图片列表生成sprite图像
def create_sprite_image(images):
    """Returns a sprite image consisting of images passed as argument. Images should be count x width x height"""
    if isinstance(images, list):
        images = np.array(images)
    img_h = images.shape[1]
    img_w = images.shape[2]
    # sprite图像可以理解成是所有小图片拼成的大正方形矩阵，大正方形矩阵中的每一个
    # 元素就是原来的小图片。于是这个正方形的地长就是sqrt(n)，其中n为小图片的数量。
    n_plots = int(np.ceil(np.sqrt(images.shape[0])))
    
    # 使用全1来初始化最终的大图片
    spriteimage = np.ones((img_h * n_plots ,img_w * n_plots ))
    
    for i in range(n_plots):
        for j in range(n_plots):
            # 计算当前图片的编号
            this_filter = i * n_plots + j
            if this_filter < images.shape[0]:
                # 将当前小图片的内容复制到最终的sprite图像
                this_img = images[this_filter]
                spriteimage[i * img_h:(i + 1) * img_h,
                  j * img_w:(j + 1) * img_w] = this_img
    
    return spriteimage

# 加载MNIST数据，这列onehot为False，得到的labels为数字
mnist = input_data.read_data_sets("../../datasets/MNIST_data", one_hot=False)

# 生成sprite图像
to_visualise = 1 - np.reshape(mnist.test.images,(-1,28,28))
sprite_image = create_sprite_image(to_visualise)

# 将生成的sprite图像放到相应的日志目录下
path_for_mnist_sprites = os.path.join(LOG_DIR, SPRITE_FILE)
plt.imsave(path_for_mnist_sprites,sprite_image,cmap='gray')
plt.imshow(sprite_image,cmap='gray')

# 生成每张图片对应的标签文件并写到相应的日志目录下。
path_for_mnist_metadata = os.path.join(LOG_DIR, META_FIEL)
with open(path_for_mnist_metadata,'w') as f:
    f.write("Index\tLabel\n")
    for index,label in enumerate(mnist.test.labels):
        f.write("%d\t%d\n" % (index,label))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.


URLError: <urlopen error [Errno 11001] getaddrinfo failed>

运行以上代码可以得到两个文件， 一个是MNIST测试数据sprite图像，这个图像包含了所有的MNIST测试数据图像。另一个是mnist_meta.tsv，下面给出了这个tsv文件的前面几行。可以看出，这个文件的第一行是每一列的说明，以后的每一行代表一张图片，在这个文件中给出了每一张图对应的真实标签。

| Index| Label |
|:----:|:-----:|
|   0 |   7  |
|   1 |   2  |
|   2 |   1  |
|   3 |   0  |
|   4 |   4  |

在生成好辅助数据之后，以下代码展示了如何使用TensorFlow代码生成PROJECTOR所需要的日志文件来可视化MNIST测试数据在最后的输出层向量。

In [ ]:
import tensorflow as tf
import mnist_inference
import os

# 加载用于生成PROJECTOR日志的帮助函数。
from tensorflow.contrib.tensorboard.plugins import projector
from tensorflow.examples.tutorials.mnist import input_data

# 和第5章中类似地定义训练模型需要的参数。这里我们同样是复用第5章中定义的
# mnist_inference过程。
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 10000
MOVING_AVERAGE_DECAY = 0.99

LOG_DIR = 'log/high_dim'
SPRITE_FILE = 'mnist_sprite.jpg'
META_FIEL = "mnist_meta.tsv"
TENSOR_NAME = "FINAL_LOGITS"

# 训练过程和第5章给出来的基本一致，唯一不同的是这里还需要返回最后测试数据经过整个
# 神经网络得到的输出层矩阵（因为有多张测试图片，每张图片对应了一个输出层向量，所以
# 返回的结果是这些向量组成的矩阵〉。
def train(mnist):
    #  输入数据的命名空间。
    with tf.name_scope('input'):
        x = tf.placeholder(tf.float32, [None, mnist_inference.INPUT_NODE], name='x-input')
        y_ = tf.placeholder(tf.float32, [None, mnist_inference.OUTPUT_NODE], name='y-input')
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    y = mnist_inference.inference(x, regularizer)
    global_step = tf.Variable(0, trainable=False)
    
    # 处理滑动平均的命名空间。
    with tf.name_scope("moving_average"):
        variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
        variables_averages_op = variable_averages.apply(tf.trainable_variables())
   
    # 计算损失函数的命名空间。
    with tf.name_scope("loss_function"):
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
        cross_entropy_mean = tf.reduce_mean(cross_entropy)
        loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    
    # 定义学习率、优化方法及每一轮执行训练的操作的命名空间。
    with tf.name_scope("train_step"):
        learning_rate = tf.train.exponential_decay(
            LEARNING_RATE_BASE,
            global_step,
            mnist.train.num_examples / BATCH_SIZE, LEARNING_RATE_DECAY,
            staircase=True)

        train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

        with tf.control_dependencies([train_step, variables_averages_op]):
            train_op = tf.no_op(name='train')
    
    # 训练模型。
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(TRAINING_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: xs, y_: ys})
                
            if i % 1000 == 0:
                print("After %d training step(s), loss on training batch is %g." % (i, loss_value))
        
        # 计算MNIST测试数据对应的输出层矩阵。
        final_result = sess.run(y, feed_dict={x: mnist.test.images})
    
    # 返回输出层矩阵的值
    return final_result


# 生成可视化最终输出层向量所需要的日志文件。
def visualisation(final_result):
    # 使用一个新的变量来保存最终输出层向量的结果。因为embedding是通过TensorFlow
    # 中变量完成的，所以PROJECTOR可视化的都是TensorFlow中的变量。于是这里要
    # 新定义一个变量来保存输出层向量的取值。
    y = tf.Variable(final_result, name=TENSOR_NAME)
    summary_writer = tf.summary.FileWriter(LOG_DIR)

    # 通过projector.ProjectorConfig类来帮助生成日志文件。
    config = projector.ProjectorConfig()
    # 增加一个需要可视化的embedding结果
    embedding = config.embeddings.add()
    # 指定这个embedding结果对应的TensorFlow变量名称
    embedding.tensor_name = y.name

    # 指定embedding结果所对应的原始数据信息。比如这里指定的就是每一张MNIST测试
    # 图片对应的真实类别。在单词向量中可以是单词ID对应的单词。这个文件是可选的，
    # 如果没有指定那么向量就没有标签。
    embedding.metadata_path = META_FIEL

    # 指定sprite图像。这个也是可选的，如果没有提供sprite图像，那么可视化的结果
    # 每一个点就是一个小圆点，而不是具体的图片。
    embedding.sprite.image_path = SPRITE_FILE
    # 在提供sprite图像时，通过single_image_dim可以指定单张图片的大小。
    # 这将用于从sprite图像中截取正确的原始图片。
    embedding.sprite.single_image_dim.extend([28,28])

    # 将PROJECTOR所需要的内容写入日志文件。
    projector.visualize_embeddings(summary_writer, config)
    
    # 生成会话，初始化新声明的变量并将需要的日志信息写入文件。
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.save(sess, os.path.join(LOG_DIR, "model"), TRAINING_STEPS)
    
    summary_writer.close()
    

# 主函数
def main(argv=None): 
    mnist = input_data.read_data_sets("../../datasets/MNIST_data", one_hot=True)
    final_result = train(mnist)
    visualisation(final_result)

if __name__ == '__main__':
    main()

*注意这一部分书上版本略有落后，可参考projector官网（https://projector.tensorflow.org)*

运行以上代码就可以得到PROJECTOR所需要的所有日志文件。在LOGDIR下启动TensorBoard就可以看到效果图。一开始不同颜色的图片（代表不同的类别）混乱地挤在一起。但是当迭代100轮之后可以明显地看出不同颜色的图片的区分度还是比较大的。

PROJECTOR界面的左上角是数据面板，有若干个选项：
- 第一个“FINAL_LOGITS”选项是选择需要可视化的Tensor，这里默认选择的是通过ProjectorConfig中指定的tensor_name，也就是名为FINAL_LOGITS的张量。点开这个选项可以看到其他可以可视化结果；
- 第二个“Label by”选项可以控制当鼠标移到一个向量上时鼠标附近显示的标签。
- 第三个“Color by”选项可以指定每一个小图片的背景颜色。

PROJECTOR界面的左下角是投影面板，提供了不同的高维向量的可视化方法，目前主要支持的就是T-SNE和PCA。无论是T-SNE还是PCA都可以将一个高维向量转化成一个低维向量，并尽量保证转化后向量中的信息不受影响。

PROJECTOR界面的右上角是检查工具面板，提供了高亮功能。例如搜索所有代表数字3的图片，所有代表数字3的图片都被高亮标出来了，而且大部分的图片都集中在一个比较小的区域，只有少数离中心区域比较远。通过这种方式可以很快地找到每个类别中比较难分的图片，加速错误案例分析的过程。